In [1]:
%tensorflow_version 2.x

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from IPython.display import clear_output

# Import the dataset

In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parents[1]))
from Helper_Functions import common_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "regression_paths", "spotify_dataset_path")

Done!


In [8]:
df_final = pd.read_csv('Final database.csv')

C:\Users\rayst\AppData\Local\Temp\ipykernel_41756\2616739251.py:1: DtypeWarning: Columns (7,8,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv('Final database.csv')


In [9]:
df_final.head()

,Country,Uri,Popularity,Title,Artist,Album/Single,Genre,Artist_followers,Explicit,Album,...,Sweden,Switzerland,Taiwan,Turkey,UK,USA,Popu_max,Top10_dummy,Top50_dummy,Cluster
0,Global,https://open.spotify.com/track/6FyRXC8tJUh863J...,31833.95,adan y eva,Paulo Londra,single,argentine hip hop,11427104.0,False,Adan y Eva,...,0,0,0,0,0,0,10,1.0,1.0,global
1,USA,https://open.spotify.com/track/6FyRXC8tJUh863J...,8.00,adan y eva,Paulo Londra,single,argentine hip hop,11427104.0,False,Adan y Eva,...,0,0,0,0,0,1,191,0.0,0.0,english speaking and nordic
2,Argentina,https://open.spotify.com/track/6FyRXC8tJUh863J...,76924.40,adan y eva,Paulo Londra,single,argentine hip hop,11427104.0,False,Adan y Eva,...,0,0,0,0,0,0,1,1.0,1.0,spanish speaking
3,Belgium,https://open.spotify.com/track/6FyRXC8tJUh863J...,849.60,adan y eva,Paulo Londra,single,argentine hip hop,11427104.0,False,Adan y Eva,...,0,0,0,0,0,0,126,0.0,0.0,english speaking and nordic
4,Switzerland,https://open.spotify.com/track/6FyRXC8tJUh863J...,20739.10,adan y eva,Paulo Londra,single,argentine hip hop,11427104.0,False,Adan y Eva,...,0,1,0,0,0,0,21,0.0,1.0,english speaking and nordic


Map the unique string values from Explicit and Album/Single column to numeric values

In [ ]:
df_final.shape

(170633, 151)

# Remove the unwanted columns

In [ ]:
# The one hot encoding is applied to some of the columns like Country, Explicit etc. We need to remove those extra columns
df_final.drop(labels=['Country', 'Genre_new'], axis=1, inplace=True)

In [ ]:
# Remove the unnecessary columns based on certain assumptions as below:
# Which seems to have no effect on the popularity of the song.
  # For example, we do not require Track_number and Track_in_album to find the popularity score of any particular song.
  # The different characteristics of a song like danceability, energy, loudness has no effect on the popularity.
# We keep either Uri or Title as unique identity of the song.
# We have Countrywise data so we can remove Cluster as well.

col_to_drop = ["Track_number", "Tracks_in_album", "Genre", "Uri", "danceability", "energy", "key", "loudness", "Explicit_false", "Explicit_true",
                "mode", "speechiness", "acoustics", "instrumentalness", "liveliness", "valence", "tempo", "duration_ms", "time_signature",
               "syuzhet_norm", "bing_norm", "afinn_norm", "nrc_norm", "syuzhet", "bing", "afinn", "nrc", "anger", "anticipation", "disgust",
               "fear", "joy", "sadness", "surprise", "trust", "negative", "positive", "n_words", "anger_norm", "anticipation_norm", "disgust_norm", "fear_norm",
               "joy_norm", "sadness_norm", "surprise_norm", "trust_norm", "negative_norm", "positive_norm", "anger_norm2", "anticipation_norm2", "disgust_norm2",
               "fear_norm2", "joy_norm2", "sadness_norm2", "surprise_norm2", "trust_norm2", "negative_norm2", "positive_norm2", "negative_bog_jr", "positive_bog_jr",
               "Bayes", "Negative_Bayes", "Neutral_Bayes", "Positive_Bayes", "LDA_Topic", "Celebrate", "Desire", "Explore", "Fun", "Hope", "Love", "Nostalgia",
               "Thug", "bing_norm_negative", "bing_norm_neutral", "bing_norm_positive", "Cluster", "Album", "album", "compilation", "single"]
df_final.drop(labels=col_to_drop, axis=1, inplace=True)

In [ ]:
# We will find the popularity Globally so we dont need the country wise data. We can drop country columns
country_col = ["Argentina", "Australia", "Austria", "Belgium", "Brazil", "Canada", "Chile", "Colombia", "Costa Rica",
               "Denmark", "Ecuador", "Finland", "France", "Germany", "Global", "Indonesia", "Ireland", "Italy",
               "Malaysia", "Mexico", "Netherlands", "New Zealand", "Norway", "Peru", "Philippines", "Poland", "Portugal",
               "Singapore", "Spain", "Sweden", "Switzerland", "Taiwan", "Turkey", "UK", "USA"]
df_final.drop(labels=country_col, axis=1, inplace=True)

In [ ]:
df_final.shape

(170633, 33)

# Pre Processing the existing columns

For Data visualization, split the dataset into 3 subsets and check the datatype in each of the feature. Some features may contain data of mixed types like int, float and string

In [ ]:
df_final.columns

Index(['Popularity', 'Title', 'Artist', 'Album/Single', 'Artist_followers',
       'Explicit', 'Release_date', 'Days_since_release', 'Released_after_2017',
       'bolero', 'boy band', 'country', 'dance/electronic', 'else', 'funk',
       'hip hop', 'house', 'indie', 'jazz', 'k-pop', 'latin', 'metal', 'opm',
       'pop', 'r&b/soul', 'rap', 'reggae', 'reggaeton', 'rock', 'trap',
       'Popu_max', 'Top10_dummy', 'Top50_dummy'],
      dtype='object')

In [ ]:
df_subset1 = df_final[['Popularity', 'Title', 'Artist', 'Album/Single', 'Artist_followers',
       'Explicit', 'Release_date', 'Days_since_release', 'Released_after_2017']]
df_subset1.shape

(170633, 9)

In [ ]:
df_subset2 = df_final[['bolero', 'boy band', 'country', 'dance/electronic',
       'else', 'funk', 'hip hop', 'house', 'indie', 'jazz', 'k-pop', 'latin', 'metal']]
df_subset2.shape

(170633, 13)

In [ ]:
df_subset3 = df_final[['opm', 'pop', 'r&b/soul', 'rap', 'reggae', 'reggaeton', 'rock',
       'trap', 'Popu_max', 'Top10_dummy', 'Top50_dummy']]
df_subset3.shape

(170633, 11)

In [ ]:
df_subset1.applymap(type).apply(pd.value_counts).fillna(0)

,Popularity,Title,Artist,Album/Single,Artist_followers,Explicit,Release_date,Days_since_release,Released_after_2017
<class 'float'>,170633.0,0.0,0.0,0.0,141961.0,0.0,0.0,170633.0,170633.0
<class 'str'>,0.0,170633.0,170633.0,170633.0,4096.0,4096.0,170633.0,0.0,0.0
<class 'int'>,0.0,0.0,0.0,0.0,24576.0,0.0,0.0,0.0,0.0
<class 'bool'>,0.0,0.0,0.0,0.0,0.0,166537.0,0.0,0.0,0.0


In [ ]:
df_subset2.applymap(type).apply(pd.value_counts).fillna(0)

,bolero,boy band,country,dance/electronic,else,funk,hip hop,house,indie,jazz,k-pop,latin,metal
<class 'int'>,170633,170633,170633,170633,170633,170633,170633,170633,170633,170633,170633,170633,170633


In [ ]:
df_subset3.applymap(type).apply(pd.value_counts).fillna(0)

,opm,pop,r&b/soul,rap,reggae,reggaeton,rock,trap,Popu_max,Top10_dummy,Top50_dummy
<class 'int'>,170633.0,170633.0,170633.0,170633.0,170633.0,170633.0,170633.0,170633.0,170633.0,0.0,0.0
<class 'float'>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170633.0,170633.0


From Above data we can clearly see that the columns Artist_followers and Explicit contain the data of mixed types

In [ ]:
df_final["Artist_followers"] = pd.to_numeric(df_final["Artist_followers"], errors='coerce')

In [ ]:
df_final["Explicit"].unique()

array([False, True, 'False', 'True', 'n-a'], dtype=object)

Here we can see that the Explicit column contains False, True and 'False', 'True'. We need to make this uniform.
Also there are 5 values of type 'n-a' which is nothing but null. As these are just 5 in numbers, we can ignore them and convert it into null values.

In [ ]:
df_final['Explicit'] = df_final['Explicit'].apply(lambda x: "'" + str(x) + "'")

In [ ]:
df_final["Explicit"].unique()

array(["'False'", "'True'", "'n-a'"], dtype=object)

In [ ]:
df_final['Explicit'].value_counts()

'False'    111131
'True'      59497
'n-a'           5
Name: Explicit, dtype: int64

In [ ]:
df_final['Explicit'] = df_final['Explicit'].map({"'False'": 0, "'True'": 1})

In [ ]:
df_final['Explicit'].value_counts()

0.0    111131
1.0     59497
Name: Explicit, dtype: int64

In [ ]:
df_final["Album/Single"].unique()

array(['single', 'album', 'compilation', 'n-a'], dtype=object)

In [ ]:
df_final["Album/Single"].value_counts()

album          93703
single         74361
compilation     2564
n-a                5
Name: Album/Single, dtype: int64

In [ ]:
df_final['Album/Single'] = df_final['Album/Single'].apply(lambda x: "'" + str(x) + "'")

In [ ]:
df_final["Album/Single"].value_counts()

'album'          93703
'single'         74361
'compilation'     2564
'n-a'                5
Name: Album/Single, dtype: int64

In [ ]:
df_final['Album/Single'] = df_final['Album/Single'].map({"'album'": 0, "'single'": 1, "'compilation'": 2})

In [ ]:
df_final["Album/Single"].value_counts()

0.0    93703
1.0    74361
2.0     2564
Name: Album/Single, dtype: int64

In [ ]:
df_final['Album/Single'].isna().sum()

5

In [ ]:
df_final.shape

(170633, 33)

In [ ]:
df_final.applymap(type).apply(pd.value_counts).fillna(0)

,Popularity,Title,Artist,Album/Single,Artist_followers,Explicit,Release_date,Days_since_release,Released_after_2017,bolero,...,pop,r&b/soul,rap,reggae,reggaeton,rock,trap,Popu_max,Top10_dummy,Top50_dummy
<class 'float'>,170633.0,0.0,0.0,170633.0,170633.0,170633.0,0.0,170633.0,170633.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170633.0,170633.0
<class 'str'>,0.0,170633.0,170633.0,0.0,0.0,0.0,170633.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
<class 'int'>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170633.0,...,170633.0,170633.0,170633.0,170633.0,170633.0,170633.0,170633.0,170633.0,0.0,0.0


# Manage Null Values

In [ ]:
df_final.isnull().sum()

Popularity                0
Title                     0
Artist                    0
Album/Single              5
Artist_followers         46
Explicit                  5
Release_date              0
Days_since_release     3222
Released_after_2017    3222
bolero                    0
boy band                  0
country                   0
dance/electronic          0
else                      0
funk                      0
hip hop                   0
house                     0
indie                     0
jazz                      0
k-pop                     0
latin                     0
metal                     0
opm                       0
pop                       0
r&b/soul                  0
rap                       0
reggae                    0
reggaeton                 0
rock                      0
trap                      0
Popu_max                  0
Top10_dummy               0
Top50_dummy               0
dtype: int64

In [ ]:
df_final.describe()

,Popularity,Album/Single,Artist_followers,Explicit,Days_since_release,Released_after_2017,bolero,boy band,country,dance/electronic,...,pop,r&b/soul,rap,reggae,reggaeton,rock,trap,Popu_max,Top10_dummy,Top50_dummy
count,170633.000000,170628.000000,1.705870e+05,170628.000000,167411.000000,167411.000000,170633.000000,170633.000000,170633.000000,170633.000000,...,170633.000000,170633.000000,170633.000000,170633.000000,170633.000000,170633.000000,170633.000000,170633.000000,170633.000000,170633.000000
mean,5417.616264,0.465861,8.509255e+06,0.348694,1337.530228,0.845667,0.001242,0.011621,0.004102,0.033704,...,0.351749,0.026431,0.079481,0.000797,0.005866,0.037138,0.009857,89.926738,0.087035,0.294843
std,13115.854526,0.528100,1.304559e+07,0.476558,2453.554101,0.361269,0.035226,0.107175,0.063918,0.180466,...,0.477518,0.160414,0.270488,0.028221,0.076368,0.189101,0.098794,57.998729,0.281887,0.455974
min,0.800000,0.000000,0.000000e+00,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,77.600000,0.000000,3.665430e+05,0.000000,428.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,40.000000,0.000000,0.000000
50%,515.150000,0.000000,2.661429e+06,0.000000,834.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,85.000000,0.000000,0.000000
75%,3867.850000,1.000000,1.157769e+07,1.000000,1240.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,138.000000,0.000000,1.000000
max,233766.900000,2.000000,7.178310e+07,1.000000,44128.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,200.000000,1.000000,1.000000


In [ ]:
df_final["Artist_followers"].fillna(2.661429e+06, inplace = True)
df_final["Days_since_release"].fillna(834.0, inplace = True)
df_final["Released_after_2017"].fillna(1.0, inplace = True)

In [ ]:
df_final.isnull().sum()

Popularity             0
Title                  0
Artist                 0
Album/Single           5
Artist_followers       0
Explicit               5
Release_date           0
Days_since_release     0
Released_after_2017    0
bolero                 0
boy band               0
country                0
dance/electronic       0
else                   0
funk                   0
hip hop                0
house                  0
indie                  0
jazz                   0
k-pop                  0
latin                  0
metal                  0
opm                    0
pop                    0
r&b/soul               0
rap                    0
reggae                 0
reggaeton              0
rock                   0
trap                   0
Popu_max               0
Top10_dummy            0
Top50_dummy            0
dtype: int64

In [ ]:
df_final.shape

(170633, 33)

In [ ]:
df_final.dropna(inplace=True)

In [ ]:
df_final.shape

(170628, 33)

In [ ]:
df_final.head()

,Popularity,Title,Artist,Album/Single,Artist_followers,Explicit,Release_date,Days_since_release,Released_after_2017,bolero,...,pop,r&b/soul,rap,reggae,reggaeton,rock,trap,Popu_max,Top10_dummy,Top50_dummy
0,31833.95,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,0,0,10,1.0,1.0
1,8.00,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,0,0,191,0.0,0.0
2,76924.40,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,0,0,1,1.0,1.0
3,849.60,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,0,0,126,0.0,0.0
4,20739.10,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,0,0,21,0.0,1.0


Assign unique numbers to Title and Artists

In [ ]:
labelencoder = preprocessing.LabelEncoder()
df_final['Title_No'] = labelencoder.fit_transform(df_final['Title'])
df_final['Artist_No'] = labelencoder.fit_transform(df_final['Artist'])

In [ ]:
df_final.head()

,Popularity,Title,Artist,Album/Single,Artist_followers,Explicit,Release_date,Days_since_release,Released_after_2017,bolero,...,rap,reggae,reggaeton,rock,trap,Popu_max,Top10_dummy,Top50_dummy,Title_No,Artist_No
0,31833.95,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,10,1.0,1.0,1334,18275
1,8.00,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,191,0.0,0.0,1334,18275
2,76924.40,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,1,1.0,1.0,1334,18275
3,849.60,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,126,0.0,0.0,1334,18275
4,20739.10,adan y eva,Paulo Londra,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,...,0,0,0,0,0,21,0.0,1.0,1334,18275


In [ ]:
len(df_final["Title"].unique()), len(df_final["Artist"].unique())

(47044, 25523)

In [ ]:
df_final.drop(labels=["Title", "Artist"], axis=1, inplace=True)

In [ ]:
df_final.head()

,Popularity,Album/Single,Artist_followers,Explicit,Release_date,Days_since_release,Released_after_2017,bolero,boy band,country,...,rap,reggae,reggaeton,rock,trap,Popu_max,Top10_dummy,Top50_dummy,Title_No,Artist_No
0,31833.95,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,0,0,...,0,0,0,0,0,10,1.0,1.0,1334,18275
1,8.00,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,0,0,...,0,0,0,0,0,191,0.0,0.0,1334,18275
2,76924.40,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,0,0,...,0,0,0,0,0,1,1.0,1.0,1334,18275
3,849.60,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,0,0,...,0,0,0,0,0,126,0.0,0.0,1334,18275
4,20739.10,1.0,11427104.0,0.0,2018-11-05,740.0,1.0,0,0,0,...,0,0,0,0,0,21,0.0,1.0,1334,18275


In [ ]:
df_final.columns.to_list()

['Popularity',
 'Album/Single',
 'Artist_followers',
 'Explicit',
 'Release_date',
 'Days_since_release',
 'Released_after_2017',
 'bolero',
 'boy band',
 'country',
 'dance/electronic',
 'else',
 'funk',
 'hip hop',
 'house',
 'indie',
 'jazz',
 'k-pop',
 'latin',
 'metal',
 'opm',
 'pop',
 'r&b/soul',
 'rap',
 'reggae',
 'reggaeton',
 'rock',
 'trap',
 'Popu_max',
 'Top10_dummy',
 'Top50_dummy',
 'Title_No',
 'Artist_No']

In [ ]:
df_final2 = df_final.groupby('Title_No').sum()
explicit_column = df_final.groupby('Title_No').first()['Explicit'].values
df_final2['Explicit'] = explicit_column
followers_column = df_final.groupby('Title_No').first()['Artist_followers'].values
df_final2['Artist_followers'] = followers_column
Days_since_release_column = df_final.groupby('Title_No').first()['Days_since_release'].values
df_final2['Days_since_release'] = Days_since_release_column
Released_after_2017_column = df_final.groupby('Title_No').first()['Released_after_2017'].values
df_final2['Released_after_2017'] = Released_after_2017_column
# for column in df_final.columns.to_list():
#   column = df_final.groupby('Title_No').first()[column].values
#   df_final2[column] = column

In [ ]:
Album_Single_column = df_final.groupby('Title_No').first()['Album/Single'].values
df_final2['Album/Single'] = Album_Single_column
Album_Single_column = df_final.groupby('Title_No').first()['Album/Single'].values
df_final2['Album/Single'] = Album_Single_column

In [ ]:
df_final2[:20]

,Popularity,Album/Single,Artist_followers,Explicit,Days_since_release,Released_after_2017,bolero,boy band,country,dance/electronic,...,r&b/soul,rap,reggae,reggaeton,rock,trap,Popu_max,Top10_dummy,Top50_dummy,Artist_No
Title_No,,,,,,,,,,,,,,,,,,,,,
0,2684.55,0.0,5502398.0,1.0,463.0,1.0,0,0,0,0,...,0,13,0,0,0,0,1485,0.0,1.0,317374
1,49.60,1.0,614607.0,1.0,876.0,1.0,0,0,0,0,...,0,0,0,0,0,0,139,0.0,0.0,19553
2,413.05,0.0,19841.0,1.0,288.0,1.0,0,0,0,0,...,0,0,0,0,0,0,68,0.0,0.0,1752
3,48.80,0.0,617156.0,1.0,212.0,1.0,0,0,0,0,...,0,0,0,0,0,0,140,0.0,0.0,24216
4,1872.70,0.0,111580.0,0.0,141.0,1.0,0,0,0,0,...,0,0,0,0,0,0,179,0.0,1.0,24909
5,408.65,1.0,697833.0,0.0,50.0,1.0,0,0,0,0,...,0,0,0,0,0,0,34,0.0,1.0,6114
6,262.40,0.0,384989.0,1.0,295.0,1.0,0,0,0,0,...,0,0,0,0,0,0,112,0.0,0.0,12986
7,64.00,1.0,252575.0,0.0,1420.0,0.0,0,0,0,0,...,0,0,0,0,0,0,172,0.0,0.0,10226
8,347.20,0.0,1157431.0,0.0,1469.0,0.0,0,0,0,0,...,0,0,0,0,0,0,147,0.0,0.0,2923


In [ ]:
# monetary_value = data.groupby(by='CustomerID')['amount'].sum()

Some of the columns contains data in different types like int, float, string mixed.
Need to maintain uniform type.

In [ ]:
# le = preprocessing.LabelEncoder()
# col_to_transform = ['Popularity', 'Days_since_release', 'Released_after_2017', 'Explicit_false', 'Explicit_true', 'bolero',
#                     'boy band', 'country', 'dance/electronic', 'else', 'funk', 'hip hop', 'house', 'indie', 'jazz', 'k-pop', 'latin', 'metal', 'opm', 'pop',
#                     'r&b/soul', 'rap', 'reggae', 'reggaeton', 'rock', 'trap', 'Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Canada', 'Chile',
#                     'Colombia', 'Costa Rica', 'Denmark', 'Ecuador', 'Finland', 'France', 'Germany', 'Global', 'Indonesia', 'Ireland', 'Italy', 'Malaysia',
#                     'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Singapore', 'Spain', 'Sweden', 'Switzerland',
#                     'Taiwan', 'Turkey', 'UK', 'USA', 'Popu_max', 'Top10_dummy', 'Top50_dummy', 'Artist_followers', 'Popu_max']
# for column in col_to_transform:
#   df_final[column] = le.fit_transform(df_final[column].astype(float))

In [ ]:
# col_to_normalize = ["Popularity", "Days_since_release", "Artist_followers", "Popu_max"]
# df_final[col_to_normalize] = df_final[col_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
# df_final.head()

In [ ]:
# df_final.shape

In [ ]:
# df_final["Release_date"].dtype

In [ ]:
# df_final.set_index('Release_date', inplace=True)

In [ ]:
# data = df_final.copy()
# data.shape

In [ ]:
# y = data.Popularity
# x = data.drop("Popularity", axis=1)

# y.shape, x.shape

In [ ]:
# y.head()

In [ ]:
# x.head()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [ ]:
# print("\nX_train:\n")
# print(X_train.head())
# print(X_train.shape)

# print("\nX_test:\n")
# print(X_test.head())
# print(X_test.shape)

# print("\ny_train:\n")
# print(y_train.head())
# print(y_train.shape)

# print("\ny_test:\n")
# print(y_test.head())
# print(y_test.shape)

In [ ]:
# CATEGORICAL_COLUMNS = ['Title', 'Artist']
# NUMERIC_COLUMNS = ["Artist_followers", "Days_since_release", "Released_after_2017", "Explicit_false", "Explicit_true", "bolero",
#                    "boy band", "country", "dance/electronic", "else", "funk", "hip hop", "house", "indie", "jazz", "k-pop", "latin", "metal",
#                    "opm", "pop", "r&b/soul", "rap", "reggae", "reggaeton", "rock", "trap", "Argentina", "Australia", "Austria", "Belgium",
#                    "Brazil", "Canada", "Chile", "Colombia", "Costa Rica", "Denmark", "Ecuador", "Finland", "France", "Germany", "Global", "Indonesia",
#                    "Ireland", "Italy", "Malaysia", "Mexico", "Netherlands", "New Zealand", "Norway", "Peru", "Philippines", "Poland", "Portugal", 
#                    "Singapore", "Spain", "Sweden", "Switzerland", "Taiwan", "Turkey", "UK", "USA", "Popu_max", "Top10_dummy", "Top50_dummy"]

# feature_columns = []
# for feature_name in CATEGORICAL_COLUMNS:
#   vocabulary = X_train[feature_name].unique()  # gets a list of all unique values from given feature column
#   feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

# for feature_name in NUMERIC_COLUMNS:
#   feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

# print(feature_columns)

In [ ]:
# def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
#   def input_function():  # inner function, this will be returned
#     ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
#     if shuffle:
#       ds = ds.shuffle(1000)  # randomize order of data
#     ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
#     return ds  # return a batch of the dataset
#   return input_function  # return a function object for use

# train_input_fn = make_input_fn(X_train, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
# eval_input_fn = make_input_fn(X_test, y_test)

In [ ]:
# X_train.head()

In [ ]:
# linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

In [ ]:
# linear_est.train(train_input_fn)  # train
# result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

In [ ]:
# clear_output()  # clears consoke output
# print(result['accuracy'])